In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

!pip install jupyter_dash

--2022-02-27 04:54:16--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2022-02-27 04:54:16 (88.8 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]

     |████████████████████████████████| 95 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 24.6 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 59.6 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=da

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11448 sha256=9492929293e6ff18c869f2ec58d0761a48b2834eb50e88474cdadd445436b90d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ce/18/7f/e9527e3e66db1456194ac7f61eb3211068c409edceecff2d31
Successfully built dash-core-components dash-html-components dash-table retrying


In [3]:
# Import required libraries
import pandas as pd
import jupyter_dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

all_sites = list(set(spacex_df['Launch Site']))

# Create a dash application
app = jupyter_dash.JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id = 'site-dropdown', options = [{'label': 'ALL', 'value': 'ALL'}] + [{'label':i, 'value':i} for i in all_sites
                                                                         ],
                                            value = 'ALL'
                                            ),
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(min_payload, max_payload, (min_payload + max_payload)/40, id = 'payload-slider')),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ],)

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))
def success_pie_chart(launch_site):
    if launch_site == 'ALL':
        total_success = spacex_df.groupby('Launch Site').sum('class')['class']
        total_fails = spacex_df.groupby('Launch Site').count()['class']- total_success

        data = {'Launch Site': total_success.index, 'Total success': total_success, 'Total fails': total_fails, 'Ratios': total_success/total_fails}
        data = pd.DataFrame(data)
        fig = px.pie(data, values='Ratios', names='Launch Site', title='Success to failure per launch site')
    else:
        total_success = spacex_df.groupby('Launch Site').sum('class')['class'].loc[launch_site]
        total_fails = spacex_df.groupby('Launch Site').count()['class'].loc[launch_site] - total_success

        data = {'Cases': ['successes', 'fails'], 'Totals': [total_success, total_fails]}
        data = pd.DataFrame(data)
        fig = px.pie(data, values='Totals', names='Cases', title='Success to failure per launch site')
    
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
              [Input(component_id = 'site-dropdown', component_property = 'value'),
               Input(component_id = 'payload-slider', component_property = 'value')])
def success_pie_chart(launch_site, payload):
    if launch_site == 'ALL':
        success = spacex_df[['class', 'Payload Mass (kg)']]
        classcat = []

        for i in success['class']:
            if i == 1:
                classcat.append('success')
            else:
                classcat.append('failure')

        success['class-categorical'] = classcat
        fig = px.scatter(success, x = 'Payload Mass (kg)', y = 'class', range_x = payload, color = 'class-categorical')
    else:
        success = spacex_df[['class', 'Payload Mass (kg)']][spacex_df['Launch Site'] == launch_site]

        classcat = []

        for i in success['class']:
            if i == 1:
                classcat.append('success')
            else:
                classcat.append('failure')

        success['class-categorical'] = classcat
        fig = px.scatter(success, x = 'Payload Mass (kg)', y = 'class', range_x = payload, color = 'class-categorical')
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()


Dash app running on http://127.0.0.1:8050/


/tmp/wsuser/ipykernel_154/4290457033.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/wsuser/ipykernel_154/4290457033.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
